In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import Counter
import os
from argparse import Namespace


In [2]:
flags = Namespace(
    train_file='note.txt',
    max_epoch = 10,
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['C', 'A'],
    predict_top_k=5,
    checkpoint_path='./checkpoint',
)


In [3]:
def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r') as f:
        text = f.read()
    text = text.split()

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))         
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

In [4]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

In [6]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

In [7]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

In [8]:
def main():
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.train_file, flags.batch_size, flags.seq_size)

    net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)
    
    if not os.path.isdir(flags.checkpoint_path):
        os.mkdir(flags.checkpoint_path)
    else:
        pass

    iteration = 0
    
    for e in range(flags.max_epoch):
        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        state_h, state_c = net.zero_state(flags.batch_size)
        
        for x, y in batches:
            iteration += 1
            
            # Tell it we are in training mode
            net.train()

            # Reset all gradients
            optimizer.zero_grad()

            x = torch.tensor(x)
            y = torch.tensor(y)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            loss.backward()

            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), flags.gradients_norm)

            optimizer.step()  

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, flags.max_epoch),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

                torch.save(net.state_dict(),
                           flags.checkpoint_path + '/model-{}.pth'.format(iteration))
                
    # generation            
    predict(net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)

In [9]:
def predict(net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)

    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]])
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    
    for _ in range(flags.seq_size):
        ix = torch.tensor([[choice]])
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

In [10]:
if __name__ == '__main__':
    main()

Vocabulary size 12
Epoch: 0/10 Iteration: 100 Loss: 2.4862728118896484
Epoch: 0/10 Iteration: 200 Loss: 2.487010955810547
Epoch: 0/10 Iteration: 300 Loss: 2.4873452186584473
Epoch: 1/10 Iteration: 400 Loss: 2.486614227294922
Epoch: 1/10 Iteration: 500 Loss: 2.488166570663452
Epoch: 1/10 Iteration: 600 Loss: 2.4853177070617676
Epoch: 2/10 Iteration: 700 Loss: 2.4885125160217285
Epoch: 2/10 Iteration: 800 Loss: 2.4850058555603027
Epoch: 2/10 Iteration: 900 Loss: 2.4846341609954834
Epoch: 3/10 Iteration: 1000 Loss: 2.4822347164154053
Epoch: 3/10 Iteration: 1100 Loss: 2.4875028133392334
Epoch: 3/10 Iteration: 1200 Loss: 2.4841675758361816
Epoch: 4/10 Iteration: 1300 Loss: 2.4836959838867188
Epoch: 4/10 Iteration: 1400 Loss: 2.4835140705108643
Epoch: 4/10 Iteration: 1500 Loss: 2.482473373413086
Epoch: 5/10 Iteration: 1600 Loss: 2.4850964546203613
Epoch: 5/10 Iteration: 1700 Loss: 2.482945680618286
Epoch: 5/10 Iteration: 1800 Loss: 2.4853804111480713
Epoch: 6/10 Iteration: 1900 Loss: 2.48625